# Data and Analysis Plan: Movie Reccomendation
## Team -23

- Vedant Bhagat (bhagat.ve@northeastern.edu)
- Anthony Lee (lee.ant@northeastern.edu)
- Dawn Lu (lu.daw@northeastern.edu)
- Josh Rosenberg (rosenberg.jo@northeastern.edu)


## Project Goal:
This work will scrape li sts f bomoviesrom [BoardGameGeek.com](https://boardgamegeek.com/) to reccmenmd a new bomovieo users who input a gamoviehey already enjoy. 

## Data 
<font color='green'>
(1%) Gives a summary of the data processing pipeline so a technical expert can easily follow along.
    
This overview section allows you to describe an overview of your data processing pipeline.  
    
Your pipeline should end by saving one or more csvs of data which will be loaded by the analysis portion of the notebook.
</font>

### Overview 
We will scrape a [list of all boardgames](https://boardgamegeek.com/browse/boardgame) ranked by popularity from BoardGameGeek.  

<img src="https://i.ibb.co/mD456p1/Screenshot-from-2021-03-14-09-25-01.png" width=800px>


From this list, for each game, we can obtain:
- title
- year published
- url to specific game page
- description (e.g. "Vanquish monsters ...")

Upon visiting an [individual game's webpage](https://boardgamegeek.com/boardgame/174430/gloomhaven) we can use the top titlebar:

<img src="https://i.ibb.co/6DdmKsg/Screenshot-from-2021-03-13-18-57-00.png" width=600px>

to observe:
- complexity rating
- playtime (mins a typical game lasts)
- min/max number of player required
- reccomended age range

Most importantly we seek to collect the category and mechanism tags associated with a particular game:
<img src="https://i.ibb.co/f1mbxVD/Screenshot-from-2021-03-13-18-56-52.png" width=150px>

Each game has multiple category tags.  Our expectation is that games with similar tags are enjoyed by similar players.  This tag data will be essential to reccomend a new game to a player who inputs another game they enjoy.  To simplify analysis, each tag will have its own column in our output DataFrame.  For example:

|       | card game | mystery | social | communication |
|-------|-----------|---------|--------|---------------|
| game0 | True      | True    | False  | False         |
| game1 | False     | True    | True   | True          |

Indicates that 
- game0 has tags 'card game' and 'mystery'
- game1 has tags ' mystery', 'social' and 'communication'

### Pipeline Overview

We will accomplish this task with three functions:
- `get_url()`
    - returns html string of a given url
- `clean_top_games()`
    - builds dataframe of [a single page of top games](https://boardgamegeek.com/browse/boardgame/page/2) from html string 
- `clean_game_meta()`
    - collects game meta data (tags, complexity, playtime, min/max players, age range ...) from an [individual game's webpage](https://boardgamegeek.com/boardgame/174430/gloomhaven)
    
As well as two short scripts:
- **Scrape list of games:** use `get_url()` and `clean_top_games()` in a loop to collect n pages of top games (100 * n games), populating a DataFrame `df_game`
- **Get meta data per game:** loop through each row of `df_game`, query and process the individual game's webpage via `get_url()` and `clean_game_meta()` using the previously collected url and append the remaining features to `df_game`

ggooooodegjsr;gnsw


### Pipeline
<font color='green'>
    
(4%) Obtains, cleans, and merges all data sources involved in the project.
    
Documentation counts!
    
The majority of this section is code, but do make sure that one can do a quick sanity check that your pipeline worked by printing a few examples (e.g. call `DataFrame.head()` a few times).
</font>


### Visualizations (sanity check / data exploration)
<font color='green'>
    
(2.5%) Builds two visualizations (graphs) from the data which characterize the distribution of the data itself in some interesting way. Your visualizations will be graded based on how much information they can effectively communicate with readers. Please make sure your visualizations are sufficiently distinct from each other.

**Wherever a non-technical reader may misunderstand, write a few sentences which specify how to interpret the graph**  It is expected that a non-technical reader can fully digest your graphs based on the images themselves as well as your explanatory text (they won't read your code).
    
Notice that the pipeline above takes 20 mins or so to complete.  We have manually re-labelled `game.csv` to `game_final.csv` so that:
- we don't overwrite our precious dataset
- we can load it below and continue on with our graphs

You won't lost any credit for not using a `_final.csv` structure in your code but I'd encourage you to do so to avoid losing your data / time :)
</font>



## Analysis Plan
<font color='green'>
    
(2%) Discuss what ML tools will be used and the relevant assumptions required to apply each. Either:

discuss why one algorithm may be chosen over the others

describe what subset of a whole suite of similar algorithms you’ll apply (its ok to say, ‘we’re going to try all of these because we don’t have reason to think one should be better than another’)

For example:

- Project goal: Predicting the movement of stock prices based on how similar companies are doing via regression.
    - This is observable: there are plenty of information you could collect about stock prices (though it will need some careful though about what a "similar" company is)
    - This is sound and simple: seems intuitive that there'd be a relationship to be found between the company's stock prices
    
- Project goal: Recommend a movie based on the events of a person's dream last night.
    - This is not observable: how will you get enough data on what people dreamed about?
    - This is not sound or simple: what kind of movies should we suggest for people with nightmares of falling?  Doesn't seem to be a clear relationship between dreams / movies a person would enjoy ....
    
    
To date, we've covered data collection and processing extensively and are only starting ML this week (Mar 22).  The grading will account for this:
- be specific / clear about which data will be used
    - give plenty of examples to illustrate things
- be as specific / clear as you can about the ML analysis
</font>

### Ovehttps://developers.themoviedb.org/3/movies/get-movie-details

In [1]:
import pandas as pd

movies = pd.read_csv('data/movies.csv')

movies.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
tags = pd.read_csv('data/tags.csv')

tags.head()

In [ ]:
ratings = pd.read_csv('data/ratings.csv')

ratings.head()

In [ ]:
links = pd.read_csv('data/links.csv')

links.head()

In [ ]:
import pandas as pd

movies = pd.read_csv('data/movies.csv')

movies.head()


In [ ]:
tags = pd.read_csv('data/tags.csv')

tags.head()

In [ ]:
ratings = pd.read_csv('data/ratings.csv')

ratings.head()

In [3]:
links = pd.read_csv('data/links.csv')

links.head()



,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Fetching data is... hard

So, it turns out that making over 60,000 API requests sequentially is prone to failure. Lots of things can go wrong - wifi can drop, laptop can fall asleep, etc.

There are a couple of solutions to this:
  - Save api results locally as you get them, so you can resume where you left off if network drops
  - Run the code doing all the fetches in a cloud vm so that it has high network reliability and stability

We ended up doing both. I spun up a small debian vm in gcp, and ran the below cell as a standalone python script. Once the script finished running, I used SCP to copy the files from the cloud vm to my local machine, so that the data was available locally for my jupyter notebook to consume. 

If this were a business use case, and we needed to run this on a regular basis, I would set this up as a cron job that ran on a scheduled basis, and dumped the data to a new s3 bucket each time it ran


In [12]:
import requests
tmdb_api_key = '433618a0549fe010dec7ca3a9dc46d53'


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def build_data_cache_from_tmdb():
    """
    Fetch all movie data from TMDB, and store it in chunked json files
    We do this because network conditions are unreliable sometimes, maybe battery dies,
    etc, so we want to be able to resume where we left off, instead of losing all the data
    we gathered. Chunks are written as json files to the /cache directory.
    """
    tmdb_id_list = links['tmdbId'].tolist()
    chunked_list = list(chunks(tmdb_id_list, 100))

    for idx, chunk in enumerate(chunked_list):
        results = []
        for movie_id in chunk:
            url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US'
            resp = requests.get(url)
            data = resp.json()
            data['tmdb_id'] = movie_id
            results.append(data)
        movies_df = pd.DataFrame(results)
        start = idx * 100
        end = idx * 100 + len(chunk)
        movies_df.to_json(f'cache/movies_{start}_{end}_data.json')


In [ ]:
# Usually I would use an environment variable to control this; However,
# Environment variables with Jupyter is tricky, so we'll just hardcode this
# boolean manually. You probably want to leave this as false and use the cache
# we've already built.
rebuild_cache = False
if rebuild_cache:
    build_data_cache_from_tmdb()



In [11]:
temp = pd.read_json('cache/movies_0_100_data.json')
temp2 = pd.read_json('cache/movies_100_200_data.json')
frames = [temp, temp2]
movies_df = pd.concat(frames)
movies_df

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,tmdb_id
0,False,/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,373554033,81,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Toy Story,False,8.0,15210,862
1,False,/dH5BdTa6iSmVJahteY0xvthBsaO.jpg,"{'id': 495527, 'name': 'Jumanji Collection', '...",65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.sonypictures.com/movies/jumanji/,8844,tt0113497,en,Jumanji,...,262797249,104,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Roll the dice and unleash the excitement!,Jumanji,False,7.2,8868,8844
2,False,/1J4Z7VhdAgtdd97nCxY7dcBpjGT.jpg,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",25000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",,15602,tt0113228,en,Grumpier Old Men,...,71500000,101,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,290,15602
3,False,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",,31357,tt0114885,en,Waiting to Exhale,...,81452156,127,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.2,120,31357
4,False,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,"{'id': 96871, 'name': 'Father of the Bride (St...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...",,11862,tt0113041,en,Father of the Bride Part II,...,76594107,106,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,6.2,564,11862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,False,None,None,0,"[{'id': 18, 'name': 'Drama'}]",,139408,tt0114534,en,The Stars Fell on Henrietta,...,0,109,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Stars Fell on Henrietta,False,6.3,13,139408
96,False,/lPvwp0esgdnb6L7ypTbgwhhAMxz.jpg,None,42000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",http://www.strange-days.com,281,tt0114558,en,Strange Days,...,7959291,145,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,You know you want it.,Strange Days,False,7.0,977,281
97,False,/wVoGdV7PGc9vs0udN2FxnCJpoqz.jpg,None,7,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,5967,tt0058450,fr,Les Parapluies de Cherbourg,...,0,91,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,A film for all the young lovers of the world.,The Umbrellas of Cherbourg,False,7.4,541,5967
98,False,/tnWk1Kued91e3OFgVoxE0bz34Oj.jpg,None,14000000,"[{'id': 53, 'name': 'Thriller'}]",,79593,tt0114666,en,The Tie That Binds,...,5830454,99,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,The Tie That Binds,False,5.2,16,79593
